In [4]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timezone

In [5]:
# ------------------------------------------------------------
# Configuration
# ------------------------------------------------------------
DB_NAME = "nba_data.db"
DB_URI = f"sqlite:///{DB_NAME}"
engine = create_engine(DB_URI, echo=False)

In [6]:
# ------------------------------------------------------------
# 1. Load Data
# ------------------------------------------------------------
query = "SELECT * FROM player_game_data;"
df = pd.read_sql(query, engine)

In [7]:
# ------------------------------------------------------------
# 2. Sort and Group by Player
# ------------------------------------------------------------
# Ensure that data is sorted by player_id and game_date to create time series features
df = df.sort_values(by=["player_id", "game_date"])

In [16]:
# ------------------------------------------------------------
# 3. Compute Trailing Averages
# ------------------------------------------------------------
# Define a trailing window size (e.g., 5 games)
window_size = 5

# Compute trailing points per 5 games
df["trailing_pts_5"] = (
    df.groupby("player_id")["pts"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)  # Reset index to align with the original DataFrame
)

# Compute trailing minutes per 5 games
df["trailing_min_5"] = (
    df.groupby("player_id")["min"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)

# Compute trailing field goals made (FGM) per 5 games
df["trailing_fgm_5"] = (
    df.groupby("player_id")["fgm"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)

# Compute trailing field goal attempts (FGA) per 5 games
df["trailing_fga_5"] = (
    df.groupby("player_id")["fga"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)

# Compute trailing rebounds per 5 games
df["trailing_reb_5"] = (
    df.groupby("player_id")["reb"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)

# Compute trailing assists per 5 games
df["trailing_ast_5"] = (
    df.groupby("player_id")["ast"]
    .apply(lambda x: x.shift(1).rolling(window=window_size, min_periods=1).mean())
    .reset_index(level=0, drop=True)
)

In [23]:
# ------------------------------------------------------------
# 4. Compute Efficiency Metrics
# ------------------------------------------------------------
# Field Goal Percentage
df["fg_pct"] = df.apply(lambda x: x["fgm"] / x["fga"] if x["fga"] > 0 else 0.0, axis=1)

# Points per Minute
df["pts_per_min"] = df.apply(lambda x: x["pts"] / x["min"] if x["min"] > 0 else 0.0, axis=1)

# Compute trailing FG% (Field Goal Percentage) over the last 5 games
df["trailing_fg_pct_5"] = (
    df.groupby("player_id", group_keys=False)
    .apply(
        lambda group: (
            group["fgm"].shift(1).rolling(window=5, min_periods=1).sum()
            / group["fga"].shift(1).rolling(window=5, min_periods=1).sum()
        ),
        include_groups=False
    )
    .fillna(0)  # Replace NaN with 0 for invalid calculations
)

# Compute trailing PPM (Points Per Minute) over the last 5 games
df["trailing_ppm_5"] = (
    df.groupby("player_id", group_keys=False)
    .apply(
        lambda group: (
            group["pts"].shift(1).rolling(window=5, min_periods=1).sum()
            / group["min"].shift(1).rolling(window=5, min_periods=1).sum()
        ),
        include_groups=False
    )
    .fillna(0)
)



In [24]:
# ------------------------------------------------------------
# . (Optional) Store Enhanced Dataset
# ------------------------------------------------------------
# Normalize column names again just to be sure
df.columns = [col.lower() for col in df.columns]

# Store this enhanced dataset into a new table
df.to_sql("player_game_features", engine, if_exists="replace", index=False)

print("Feature engineering complete. Enhanced data stored in 'player_game_features' table.")

Feature engineering complete. Enhanced data stored in 'player_game_features' table.


In [25]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", engine)

,name
0,player_game_data
1,player_game_features
